In [1]:
import numpy as np
from itertools import accumulate
from math import sqrt
np.set_printoptions(precision=5, suppress=True)

In [2]:
lista=[[1,2],[4,4],[-1,3],[16,22],[-42,0],[22,-21]]

In [3]:
homo_lista=[[7,4,1],[6,4,2],[5,2,0.5],[0,0,10],[-5,-5,1]]

In [4]:
trapez1 = [[- 3, - 1, 1], [3, - 1, 1], [1, 1, 1], [- 1, 1, 1], [1,2,3], [-8,-2,1]] 
pravougaonik1 = [[- 2, - 1, 1], [2, - 1, 1], [2, 1, 1], [- 2, 1, 1], [2,1,5], [-16,-5,5]]

In [5]:
orig=[[-3,-1,1],[3,-1,1],[1,1,1],[-1,1,1]]
slike=[[-2,-1,1],[2,-1,1],[2,1,1],[-2,1,1]]

In [6]:
def afinizuj(niz):
    return list(map(lambda x : [x[0]/x[2],x[1]/x[2]], niz))

In [7]:
def homo(niz):
    return list(map(lambda x : [x[0],x[1],1], niz))

Normalizacija tacaka za odredjivanje matrice projektivnog preslikavanja.

In [8]:
def normMatrix(lista):
    lista1=np.array(afinizuj(lista))
    teziste1=(list(accumulate(lista1))[-1])/(len(lista))
    lista2=np.array(lista1-teziste1)
    inteziteti=np.array(list(map(lambda x: [sqrt(x[0]*x[0]+x[1]*x[1])],lista2)))
    d=(list(accumulate(inteziteti))[-1])/(len(lista))
    translacija=np.array([[1,0,0],[0,1,0],np.append(-teziste1,1)])
    translacija=translacija.transpose()
    homotetija=np.array([[sqrt(2)/float(d),0,0],[0,sqrt(2)/float(d),0],[0,0,1]])
    transform=np.matmul(homotetija,translacija)
    nova_lista=[]
    for x in np.array(lista):
        x=x.reshape(3,1)
        x=np.matmul(transform,x)
        nova_lista.append([x[0,0],x[1,0],x[2,0]])
    return nova_lista,transform

Normiranje matrice

In [9]:
def norm_matrix(matrica):
    return matrica/matrica[2,2]

DLT i DLT sa normalizacijom

In [10]:
def DLT(originali,slike):
    matrica=[]
    for x,y in zip(originali,slike):
        prvi=[0,0,0,-y[2]*x[0],-y[2]*x[1],-y[2]*x[2],y[1]*x[0],y[1]*x[1],y[1]*x[2]]
        drugi=[y[2]*x[0],y[2]*x[1],y[2]*x[2],0,0,0,-y[0]*x[0],-y[0]*x[1],-y[0]*x[2]]
        matrica.append(prvi)
        matrica.append(drugi)
    matrica=np.array(matrica)
    U,D,VT=np.linalg.svd(matrica)
    p=VT[-1]
    return norm_matrix(p.reshape(3,3))

In [11]:
def DLTwithNormalization(originali,slike):
    originali,T=normMatrix(originali)
    slike,T1=normMatrix(slike)
    p=DLT(originali,slike)
    p=np.linalg.inv(T1)@p@T
    return norm_matrix(p)

Implementacija naivnog algoritma za odredjivanje matrice projektivnog preslikavanja.

In [12]:
def opsti_polozaj(lista):
    x1, x2, x3, x4 = lista[0], lista[1], lista[2], lista[3]
    if min(np.abs([np.linalg.det([x1, x2, x3]),
                      np.linalg.det([x2, x3, x4]),
                      np.linalg.det([x3, x4, x1]),
                      np.linalg.det([x4, x1, x2])])) == 0:
        return False
    return True

def naivni(originali,slike):
    if not opsti_polozaj(originali):
        return "Losi originali!"
    if not opsti_polozaj(slike):
        return "Lose slike!"
    b=originali.pop(3)
    b1=slike.pop(3)
    x=np.linalg.solve(np.asmatrix(originali).transpose(),b)
    x1=np.linalg.solve(np.asmatrix(slike).transpose(),b1)
    x=np.array(list(map(lambda z:[z],x)))
    x1=np.array(list(map(lambda z:[z],x1)))
    p1=np.column_stack(originali*x)
    p2=np.column_stack(slike*x1)
    p=p2@(np.linalg.inv(p1))
    for i in range(0,3):
        p[i,]=list(map(lambda x: 0 if abs(x)<1e-10 else x,p[i,]))
    return norm_matrix(p)

Deo sa interfejsom koji implementira neku od funkcija za odredjivanje matrice projektivnog preslikavanja i transformise tacke zadate klikom na interfejsu, radi otklanjanja projektivne distorzije.

In [ ]:
import cv2 as cv
import numpy as np
import math

putanja = './karta.jpg'
slika = cv.imread(putanja)

tacke = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac,tacke
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, str(x) + ',' +
                    str(y), (x,y), font, 
                    1, (255, 0, 0), 2) 
        cv.imshow("karta", slika)
        
        tacke.append([x, y, 1])
        brojac += 1
        
        if brojac == 4:
            odnos=1.667
            visina=math.sqrt((tacke[2][0]-tacke[1][0])*(tacke[2][0]-tacke[1][0])+(tacke[2][1]-tacke[1][1])*(tacke[2][1]-tacke[1][1]))
            sirina=odnos*visina;
            tacke2 = np.float32([[tacke[0][0],tacke[0][1],1], [tacke[0][0]+sirina, tacke[0][1],1], [tacke[0][0]+sirina, tacke[0][1]+visina,1], [tacke[0][0], tacke[0][1]+visina,1]])
            #P=DLTwithNormalization(list(tacke),list(tacke2))
            P=naivni(list(tacke),list(tacke2))
            offset=200
            trans=np.zeros((int(sirina+offset),int(visina+offset)),dtype=np.uint8)
            out=cv.warpPerspective(slika,P,(slika.shape[1],slika.shape[0]),flags=cv.INTER_LINEAR)
            cv.imshow("izlaz",out)
            

cv.imshow("karta", slika)

cv.setMouseCallback("karta", click_event)

cv.waitKey(0)
cv.destroyAllWindows()